### setup

In [1]:
%load_ext lab_black

In [2]:
import requests
import urllib.request
import numpy as np
import datetime
import random

In [3]:
import pandas as pd

# import gspread
# from oauth2client.service_account import ServiceAccountCredentials

In [4]:
import altair as alt
import altair_grid as altgrid

alt.themes.register("grid", altgrid.theme)
alt.themes.enable("grid")

ThemeRegistry.enable('grid')

In [5]:
from datawrapper import Datawrapper

dw = Datawrapper(
    access_token="FtIwtvFtoGLaRT9a3gjX69PLu4wSuRyKddoOz6SOPw3k9wWyNICMHTkcPhOGCR5Z"
)

In [6]:
# scope = ["https://spreadsheets.google.com/feeds"]
# credentials = ServiceAccountCredentials.from_json_keyfile_name(
#    "jupyter-integration-credentials.json", scope
# )
# gc = gspread.authorize(credentials)

In [7]:
# spreadsheet_key = "1_RXzXkHPEyDAiDKmz98wTSC9UNo-8OyjGTT540X_vJk"
# book = gc.open_by_key(spreadsheet_key)

In [8]:
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 1000

In [9]:
# Opening the worksheet by using Worksheet ID
# workbook = gc.open_by_key(spreadsheet_key)
# Selecting which sheet to pulling the data
# sheet = workbook.worksheet("Sheet1")
# Pulling the data and transform it to the data frame
# values = sheet.get_all_values()

### read in data

In [10]:
df = pd.read_excel(
    "data/raw/vsrr023-tables2.xlsx", sheet_name="Table 44", skiprows=2, skipfooter=1
)

In [11]:
raw_le = pd.read_csv("data/raw/NCHS_-_Death_rates_and_life_expectancy_at_birth.csv")

#### clean data 2000-2021

In [12]:
df = df[df.columns.drop(list(df.filter(regex="Unnamed")))]

In [13]:
df.columns = [
    "year",
    "overall",
    "male",
    "female",
    "hispanic",
    "hispanic_male",
    "hispanic_female",
    "aian",
    "aian_male",
    "aian_female",
    "aapi",
    "aapi_male",
    "aapi_female",
    "black",
    "black_male",
    "black_female",
    "white",
    "white_male",
    "white_female",
]

In [14]:
df = df.drop(index=6).reset_index()

In [15]:
df = df[~df["overall"].isna()].drop(columns="index")

In [16]:
df["year"] = df["year"].str[:4]

In [17]:
years = ["2019", "2020", "2021"]

#### clean data 1900-2018

In [18]:
raw_le.columns = (
    raw_le.columns.str.lower()
    .str.replace(" ", "_")
    .str.replace(r"\(.*\)", "", regex=True)
)

In [19]:
le_1900_slim = raw_le[
    (raw_le["race"] == "All Races") & (raw_le["sex"] == "Both Sexes")
][["year", "average_life_expectancy_"]]

In [20]:
df_1900_99 = le_1900_slim.drop(le_1900_slim.loc[100:118].index).rename(
    columns={"average_life_expectancy_": "overall"}
)

#### clean data 2000-2021

In [21]:
df_overall = df.drop(list(df)[2:19], axis=1)

In [22]:
df_overall = df_overall.sort_values("year", ascending=True)

#### combine the two datasets

In [23]:
final = pd.concat([df_1900_99, df_overall])

#### make a chart (or add data in)